In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
%%time
pred1 = dd.read_csv('test_pred_27.csv')
pred2 = dd.read_csv('test_pred_28.csv')
pred1 = pred1.compute()
pred2 = pred2.compute()
print(pred1.shape,pred2.shape)

(3492890, 16) (3492890, 16)
CPU times: user 34 s, sys: 3.26 s, total: 37.3 s
Wall time: 9.37 s


In [4]:
%%time
pred1 = pred1.reset_index(drop=True)
pred2 = pred2.reset_index(drop=True)

CPU times: user 223 ms, sys: 403 ms, total: 626 ms
Wall time: 629 ms


In [5]:
pred1['object_id'].equals(pred2['object_id'])

True

In [6]:
%%time
test_ensemble = pd.DataFrame()

CPU times: user 395 µs, sys: 50 µs, total: 445 µs
Wall time: 451 µs


In [7]:
coef_lgb = 0.8
coef_mlp = 0.2

In [8]:
%%time
test_ensemble = pred1.iloc[:,1:]*coef_lgb + pred2.iloc[:,1:]*coef_mlp

CPU times: user 369 ms, sys: 790 ms, total: 1.16 s
Wall time: 1.17 s


In [9]:
print(test_ensemble.shape)

(3492890, 15)


In [10]:
test_ensemble['object_id'] = pred1['object_id']
print(test_ensemble.shape)

(3492890, 16)


In [11]:
test_ensemble = test_ensemble[list(pred1.columns)]

In [12]:
test_ensemble.tail()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
3492885,130787966,0.000260,0.319900,0.000442,0.211384,0.165349,0.000498,0.118727,0.000217,0.002128,0.019701,0.002872,0.157412,0.000333,0.000777,0.247016
3492886,130787971,0.000190,0.011871,0.000255,0.050266,0.198113,0.000295,0.019613,0.349657,0.000475,0.027096,0.000895,0.338094,0.000283,0.002898,0.239185
3492887,130787974,0.000597,0.020802,0.000146,0.856617,0.034148,0.000317,0.068468,0.000054,0.000165,0.001077,0.000448,0.014393,0.000201,0.002568,0.093202
3492888,130788053,0.000076,0.703793,0.000152,0.138461,0.005300,0.000154,0.010002,0.101810,0.000470,0.000332,0.000199,0.038139,0.000164,0.000948,0.170398
3492889,130788054,0.000230,0.097277,0.000340,0.375996,0.207893,0.000515,0.053825,0.000138,0.000718,0.018535,0.000935,0.242125,0.000685,0.000788,0.237045


In [13]:
%%time
test_ensemble.to_csv('test_pred_30.csv',index=False)

CPU times: user 1min 31s, sys: 1.32 s, total: 1min 32s
Wall time: 1min 33s


In [14]:
test_ensemble['object_id'].equals(pred1['object_id'])

True

In [15]:
!kaggle competitions submit -c PLAsTiCC-2018 -f test_pred_30.csv -m "Message"

100%|█████████████████████████████████████| 1.06G/1.06G [2:53:20<00:00, 110kB/s]
Successfully submitted to PLAsTiCC Astronomical Classification